In [1]:
import numpy as np
import matplotlib.pyplot as plt

---
#### Dependencies

Numpy und matplotlib werden benötigt.
Folgende Module sind nur für die interaktive Visualisierung nötig. 

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

Installation unter Jupyter Notebook:   
``` > conda install -c conda-forge ipywidgets ``` 

Installation unter Jupyter Lab:  
``` > conda install -c conda-forge nodejs  ```  
``` > conda install -c conda-forge ipywidgets   ```   
``` > jupyter labextension install @jupyter-widgets/jupyterlab-manager```   

---

Funktionen aus Übung 2

In [3]:
def f(x):
    return 3*x + np.e**(-2*x**2) + 3*np.sin(x)

In [4]:
def df(x):
    return 3 + np.e**(-2*x**2)*(-4*x) + 3*np.cos(x)

In [5]:
def g1(x, y):
    return f(x) + x

In [6]:
def g2(x, a):
    return a*f(x) + x

---

#### Implementation

In [7]:
def fpi(start, a=-0.1497299, inverse_zoom = 0.2, iterations = 5):
    path = [(start, g2(start, a))]
    for _ in range(iterations):
        path.append((path[-1][1], path[-1][1]))
        path.append((path[-1][1], g2(path[-1][1], a)))
        
    lengthx = max([x[0] for x in path]) - min([x[0] for x in path])
    lengthy = max([x[1] for x in path]) - min([x[1] for x in path])
    minboundsx = min([x[0] for x in path]) - inverse_zoom * max(lengthx, lengthy)
    maxboundsx = max([x[0] for x in path]) + inverse_zoom * max(lengthx, lengthy)
    minboundsy = min([x[1] for x in path]) - inverse_zoom * max(lengthx, lengthy)
    maxboundsy = max([x[1] for x in path]) + inverse_zoom * max(lengthx, lengthy)
    length = max(lengthy, lengthx)
    if lengthx < lengthy:
        minboundsx -= (lengthy - lengthy) / 2
        maxboundsx += (lengthy - lengthy) / 2
    else:
        minboundsy -= (lengthx - lengthy) / 2
        maxboundsy += (lengthx - lengthy) / 2
    
    S = np.linspace(minboundsx, maxboundsx, 200)
    plt.figure(figsize=(8,8))
    plt.plot([minboundsx, maxboundsx], [minboundsx, maxboundsx], ls="--", label="Bisection")
    plt.plot(S, [a*f(s)+s for s in S])
    plt.xlim((minboundsx, maxboundsx))
    plt.ylim((minboundsy, maxboundsy))
    for x, y, z in zip(path[::2], path[1::2], path[2::2]):
        plt.plot([x[0], y[0]], [x[1], y[1]], c="green")
        plt.plot([y[0], z[0]], [y[1], z[1]], c="green", ls="--")
    plt.annotate("start", (path[0][0]+0.01*length, path[0][1]+0.01*length))
    plt.show()

In [8]:
def newton(start, f, df, iterations = 5, inverse_zoom = 0.2):
    X = []
    def x(i):
        if i == 0:
            X.append((start, f(start)))
            return start
        xi = x(i-1)
        xi = xi - (f(xi)/df(xi))
        X.append((xi, 0))
        X.append((xi, f(xi)))
        return xi
    x(iterations)
    
    lengthx = max([x[0] for x in X]) - min([x[0] for x in X])
    lengthy = max([x[1] for x in X]) - min([x[1] for x in X])
    minboundsx = min([x[0] for x in X]) - inverse_zoom*lengthx
    maxboundsx = max([x[0] for x in X]) + inverse_zoom*lengthx
    minboundsy = min([x[1] for x in X]) - inverse_zoom*lengthy
    maxboundsy = max([x[1] for x in X]) + inverse_zoom*lengthy
    length = max(lengthy, lengthx)
    
    
    S = np.linspace(minboundsx, maxboundsx, 200)
    plt.figure(figsize=(8,8))
    plt.axhline(0, ls='--')
    plt.plot(S, [f(s) for s in S], c="red")
    plt.xlim((minboundsx, maxboundsx))
    plt.ylim((minboundsy, maxboundsy))
    for x, y, z in zip(X[::2], X[1::2], X[2::2]):
        plt.plot([x[0], y[0]], [x[1], y[1]], c="green")
        plt.plot([y[0], z[0]], [y[1], z[1]], c="green", ls="--")
    plt.annotate("start", (X[0][0]+0.01*length, X[0][1]+0.01*length))
    plt.show()

---
### Interaktiver Bereich

---
FPI für die Funktion aus Übung 2

In [9]:
_ = interact(fpi, start=(-1.95, 2.05), a=(-1.05, 1.05), inverse_zoom=(0., 2.), iterations = (1, 20))

interactive(children=(FloatSlider(value=0.050000000000000044, description='start', max=2.05, min=-1.95), Float…

---
##### Newton Verfahren für f(x) = sin(x)

In [10]:
_ = interact(newton, start=(-1.95, 2.05, 0.1), inverse_zoom = (0., 2.), f=fixed(lambda x: np.sin(2*x)), df = fixed(lambda x: 2*np.cos(2*x)))

interactive(children=(FloatSlider(value=0.050000000000000044, description='start', max=2.05, min=-1.95), IntSl…

---
##### Newton Verfahren für Funktion aus Übung 2

In [11]:
_ = interact(newton, start=(-1.95, 2.05), inverse_zoom = (0., 2.), f=fixed(f), df = fixed(df))

interactive(children=(FloatSlider(value=0.050000000000000044, description='start', max=2.05, min=-1.95), IntSl…

---
##### Newton Verfahren für f(x) = tan(x)

In [12]:
_ = interact(newton, start=(-1.95, 2.05), inverse_zoom = (0., 2.), f=fixed(lambda x: np.tan(x)), df = fixed(lambda x: (1/np.cos(x))**2))

interactive(children=(FloatSlider(value=0.050000000000000044, description='start', max=2.05, min=-1.95), IntSl…

---
##### Anwendung der Newton Methode für eigene Funktionen
Führe folgende Zeile aus. Ersetze dabei START_STELLE durch den Bereich, an welchem sich die Startstelle befinden kann (format: (anfang, ende, schrittweite)), FUNKTION durch eine Referenz auf eine eigene Funktion und ABLEITUNG auf deren Ableitung (letzteres ist nötig, da das Program die Ableitung nicht selbst bestimmen kann).   
``` _ = interact(newton, start=START_STELLE, inverse_zoom = (0., 2.), f=fixed(FUNKTION), df = fixed(ABLEITUNG)) ```

---
_made by Lennart Mischnaewski and Dennis Hoebelt_